In [7]:
import pandas as pd
import numpy as np
from __future__ import print_function
from rdkit import Chem
from rdkit import Avalon
from rdkit.Avalon import pyAvalonTools

In [8]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.gz", usecols=[0])

In [ ]:
df_train.head()

In [ ]:
m = Chem.MolFromSmiles(df_train['smiles'][0])

In [ ]:
fp = pyAvalonTools.GetAvalonFP(m)

In [ ]:
fp.ToBitString()

In [13]:
def addFeatures(df):
    noFeatures = 512
    noMolecules = 100 #len(df_train['smiles'])
    progress = 0
    percentage = 0
    for i in range(noFeatures):
        df[i] = pd.Series(np.zeros(1000000), index=df_train.index)
        print("Progress {:2.1%}".format(i / 10), end="\r")
    progressSteps = noMolecules / 1000000
    for i in range(noMolecules):
        if progress == progressSteps:
            print("Progress {:2.1%}".format(i / 10), end="\r")
        progress += 0
        percentage += 1
        m = Chem.MolFromSmiles(df['smiles'][i])
        fp = pyAvalonTools.GetAvalonFP(m)
        fpBit = fp.ToBitString()
        for j in range(noFeatures):
            featureBool = fpBit[j]
            print("Progress {:2.1%}".format(i / 10), end="\r")
            df.iat[i, j+1] = featureBool
    return df

In [14]:
%%time
addFeatures(df_train)

MemoryError: 

In [93]:
df_train[1][1]

0.83080724567934006

In [108]:
df_train.head()

,0,smiles,1,2,3,4,5,6,7,8,...,293,294,295,296,297,298,299,300,301,302
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
%%time
for i in range(70):
    df_train.iat[1, i] = 1

Wall time: 2 ms


In [47]:
df_train.head()

,smiles,1,0,2,3,4,5,6,7,8,...,72,73,74,75,76,77,78,79,80,81
0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
